# main_with_inception_module <hr/>


### How looks Google Inception Module ?
<img src="./inception_implement.png">

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import random

C:\Users\che99\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#import and cut datas
train_data = pd.read_csv('fashion-mnist_train.csv', dtype='float32')
train_data = np.array(train_data)
train_Y = train_data[:,[0]]
train_X = train_data[:,1:]

test_data = pd.read_csv('fashion-mnist_test.csv', dtype='float32')
test_data = np.array(test_data)
test_Y = test_data[:,[0]]
test_X = test_data[:,1:]

In [3]:
class BatchGenerator():
    where = 0
    def __init__(self, x, y, batch_size, one_hot = False, nb_classes = 0):
        self.nb_classes = nb_classes
        self.one_hot = one_hot
        self.x_ = x
        self.y_ = y
        self.batch_size = batch_size
        
        self.total_batch = int(len(x) / batch_size)
        self.x = self.x_[:batch_size,:]
        self.y = self.y_[:batch_size,:]
        self.where = batch_size
        
        if self.one_hot :
            self.set_one_hot()

    def next_batch(self):
        if self.where + self.batch_size > len(self.x_) :
            self.where = 0
            
        self.x = self.x_[self.where:self.where+self.batch_size,:]
        self.y = self.y_[self.where:self.where+self.batch_size,:]
        self.where += self.batch_size
        
        if self.one_hot:
            self.set_one_hot()
        
    def set_one_hot(self):
        self.y = np.int32(self.y)
        one_hot = np.array(self.y).reshape(-1)
        self.y = np.eye(self.nb_classes)[one_hot].astype(np.int)

In [4]:
X = tf.placeholder(shape=([None, 784]), dtype=tf.float32)
X_img = tf.reshape(X, shape=[-1,28,28,1])
Y = tf.placeholder(shape=([None, 10]), dtype=tf.float32)

In [5]:
def create_weight(size, name):
    return tf.Variable(tf.random_normal(size, stddev=0.01), name=name)

def create_bias(size,name):
    return tf.Variable(tf.random_normal(size, stddev=0.01), name=name)

def conv2d(x,W):
    return tf.nn.conv2d(x,W, strides=[1,1,1,1], padding='SAME')

def max_pooling(x, size):
    return tf.nn.max_pool(x, ksize=size, strides=[1,1,1,1], padding='SAME')

In [14]:
map1 = 32
map2 = 64
num_fc1 = 700
num_fc2 = 10
reduce1x1 = 16
dropout = 0.7

#### Inception module1

In [7]:
# followings input
W_1x1_conv1_1 = create_weight([1,1,1,reduce1x1], 'W_1x1_conv1_1')
b_1x1_conv1_1 = create_bias([reduce1x1], 'b_1x1_conv1_1')

W_1x1_conv1_2 = create_weight([1,1,1,reduce1x1], 'W_1x1_conv1_2')
b_1x1_conv1_2 = create_bias([reduce1x1], 'b_1x1_conv1_2')

W_1x1_conv1_3 = create_weight([1,1,1,map1], 'W_1x1_conv1_3')
b_1x1_conv1_3 = create_bias([map1], 'b_1x1_conv1_3')

# followings 1x1
W_3x3_conv1 = create_weight([3,3,reduce1x1,map1], 'W_3x3_conv1')
b_3x3_conv1 = create_bias([map1], 'b_3x3_conv1')

W_5x5_conv1 = create_weight([5,5,reduce1x1,map1], 'W_5x5_conv1')
b_5x5_conv1 = create_bias([map1], 'b_5x5_conv1')


# followings max pooling
W_1x1_conv1_4 = create_weight([1,1,1,map1], 'W_1x1_conv1_4')
b_1x1_conv1_4 = create_bias([map1], 'b_1x1_conv1_4')

#### Inception module2

In [8]:
# followings Inception1
W_1x1_conv2_1 = create_weight([1,1,4*map1,reduce1x1], 'W_1x1_conv2_1')
b_1x1_conv2_1 = create_bias([reduce1x1], 'b_1x1_conv2_1')

W_1x1_conv2_2 = create_weight([1,1,4*map1,reduce1x1], 'W_1x1_conv2_2')
b_1x1_conv2_2 = create_bias([reduce1x1], 'b_1x1_conv2_2')

W_1x1_conv2_3 = create_weight([1,1,4*map1,map2], 'W_1x1_conv2_3')
b_1x1_conv2_3 = create_bias([map2], 'b_1x1_conv2_3')


# followings 1x1
W_3x3_conv2 = create_weight([3,3,reduce1x1,map2], 'W_3x3_conv2')
b_3x3_conv2 = create_bias([map2], 'b_3x3_conv2')

W_5x5_conv2 = create_weight([5,5,reduce1x1,map2], 'W_5x5_conv2')
b_5x5_conv2 = create_bias([map2], 'b_5x5_conv2')


# followings max pooling
W_1x1_conv2_4 = create_weight([1,1,4*map1,map2], 'W_1x1_conv2_4')
b_1x1_conv2_4 = create_bias([map2], 'b_1x1_conv2_4')

#Fully connected layers
W_fc1 = create_weight([28*28*(4*map2),num_fc1], 'W_fc1')
b_fc1 = create_weight([num_fc1], 'b_fc1')

W_fc2 = create_weight([num_fc1, num_fc2], 'W_fc2')
b_fc2 = create_weight([num_fc2], 'b_fc2')

In [9]:
def model(x, train=True):
    # inception module 1 
    conv1_1x1_1 = tf.nn.relu(conv2d(X_img, W_1x1_conv1_1) + b_1x1_conv1_1)
    conv1_1x1_2 = tf.nn.relu(conv2d(X_img, W_1x1_conv1_2) + b_1x1_conv1_2)
    maxpool1 = max_pooling(X_img,[1,3,3,1])

    conv1_3x3 = conv2d(conv1_1x1_1, W_3x3_conv1) + b_3x3_conv1
    conv1_5x5 = conv2d(conv1_1x1_2, W_5x5_conv1) + b_5x5_conv1
    conv1_1x1_3 = conv2d(X_img, W_1x1_conv1_3) + b_1x1_conv1_3
    conv1_1x1_4 = conv2d(maxpool1, W_1x1_conv1_4) + b_1x1_conv1_4
    
    inception1 = tf.nn.relu(tf.concat([conv1_3x3, conv1_5x5, conv1_1x1_3, conv1_1x1_4], 3))

    # inception module 2
    conv2_1x1_1 = tf.nn.relu(conv2d(inception1, W_1x1_conv2_1) + b_1x1_conv2_1)
    conv2_1x1_2 = tf.nn.relu(conv2d(inception1, W_1x1_conv2_2) + b_1x1_conv2_2)
    maxpool2 = max_pooling(inception1,[1,3,3,1])

    conv2_3x3 = conv2d(conv2_1x1_1, W_3x3_conv2) + b_3x3_conv2
    conv2_5x5 = conv2d(conv2_1x1_2, W_5x5_conv2) + b_5x5_conv2
    conv2_1x1_3 = conv2d(inception1, W_1x1_conv2_3) + b_1x1_conv2_3
    conv2_1x1_4 = conv2d(maxpool2, W_1x1_conv2_4) + b_1x1_conv2_4
    
    inception2 = tf.nn.relu(tf.concat([conv2_3x3, conv2_5x5, conv2_1x1_3, conv2_1x1_4], 3))
    
    inception2_flat = tf.reshape(inception2, [-1, 28*28*4*map2])
    
    if train:
        h_fc1 = tf.nn.dropout(tf.nn.relu(tf.matmul(inception2_flat, W_fc1) + b_fc1), dropout)
        
    else:
        h_fc1 = tf.nn.relu(tf.matmul(inception2_flat, W_fc1) + b_fc1)
        
    return tf.matmul(h_fc1, W_fc2) + b_fc2

In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model(X), labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [11]:
correct_prediction = tf.equal(tf.argmax(model(X,train=False), 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [17]:
epoch = 10
print('Started!')

batch = BatchGenerator(batch_size=100, one_hot=True, nb_classes=10, x=train_X, y=train_Y)
for i in range(epoch):
    avg_cost = 0
    for b in range(batch.total_batch):
        c, _ = sess.run([cost, optimizer], feed_dict={X:batch.x, Y:batch.y})
        batch.next_batch()
        avg_cost += c / batch.total_batch
    print('epoch:', i, 'cost:', avg_cost)

Started!
epoch: 0 cost: 0.010257292232830275
epoch: 1 cost: 0.009854835714704677
epoch: 2 cost: 0.009519968060127811
epoch: 3 cost: 0.007952607767698894
epoch: 4 cost: 0.006007270039578239
epoch: 5 cost: 0.006704278422463779
epoch: 6 cost: 0.007705514641684205
epoch: 7 cost: 0.006935674203112211
epoch: 8 cost: 0.006242332361783463
epoch: 9 cost: 0.004668588463887319


In [18]:
print('----- TRAIN DATA ACCURACY -----')
accu  = 0
batch = BatchGenerator(train_X, train_Y, batch_size=100, nb_classes=10,one_hot=True)
for i in range(batch.total_batch):
    feed_dict = {X:batch.x, Y:batch.y}
    accu += sess.run(accuracy, feed_dict=feed_dict)
    batch.next_batch()

print( (accu / batch.total_batch) * 100 , '%' )


print('----- TEST DATA ACCURACY -----')
accu  = 0
test = BatchGenerator(test_X, test_Y, batch_size=100 ,one_hot=True, nb_classes=10)
for i in range(test.total_batch):
    feed_dict = {X:test.x, Y:test.y}
    accu += sess.run(accuracy, feed_dict=feed_dict)
    test.next_batch()

print( (accu / test.total_batch) * 100 , '%' )

----- TRAIN DATA ACCURACY -----
99.8583334684372 %
----- TEST DATA ACCURACY -----
92.59000039100647 %
